# Python Expressions

.. note::
    This page is still in progress. More examples and detailed explanation coming soon

In [1]:
import pandas as pd
from excelbird import *
import excelbird as xb
PATH = "test.xlsx"

In [2]:
df = pd.DataFrame(
    zip([1,2,3,4], ['one', 'two', 'three', 'four']),
    columns=['Number', 'Word']
)

fr = Frame(df)
fr

Number,Word
Cell(1),Cell(one)
Cell(2),Cell(two)
Cell(3),Cell(three)
Cell(4),Cell(four)


Excelbird will track 'cell references' on its own as object references, and later convert them to real cell locations when the Book is written

In [3]:
fr['Big Number'] = fr['Number'] + " thousand"
fr

Number,Word,Big Number
Cell(1),Cell(one),Cell({...})
Cell(2),Cell(two),Cell({...})
Cell(3),Cell(three),Cell({...})
Cell(4),Cell(four),Cell({...})


In [ ]:
xb.Book(fr).write(PATH)

<img src="https://i.imgur.com/PuxeEGQ.png" width="300"/>

In [5]:
from excelbird.fn import CONCAT  # All 506 builtin excel functions here

fr = Frame(df)
fr['Big Number'] = fr['Number'] + " thousand"

fr['Combined'] = CONCAT( fr.loc['Word':'Big Number'] )

Book(fr).write(PATH)

<img src="https://i.imgur.com/XOXLZbv.png" width="400"/>